# Brain Stroke Analysis with SQL

### About Dataset

A stroke is a medical condition in which poor blood flow to the brain causes cell death. There are two main types of stroke: ischemic, due to lack of blood flow, and hemorrhagic, due to bleeding. Both cause parts of the brain to stop functioning properly. Signs and symptoms of a stroke may include an inability to move or feel on one side of the body, problems understanding or speaking, dizziness, or loss of vision to one side. Signs and symptoms often appear soon after the stroke has occurred. If symptoms last less than one or two hours, the stroke is a transient ischemic attack (TIA), also called a mini-stroke. A hemorrhagic stroke may also be associated with a severe headache. The symptoms of a stroke can be permanent. Long-term complications may include pneumonia and loss of bladder control.

The main risk factor for stroke is high blood pressure. Other risk factors include high blood cholesterol, tobacco smoking, obesity, diabetes mellitus, a previous TIA, end-stage kidney disease, and atrial fibrillation. An ischemic stroke is typically caused by blockage of a blood vessel, though there are also less common causes. A hemorrhagic stroke is caused by either bleeding directly into the brain or into the space between the brain's membranes. Bleeding may occur due to a ruptured brain aneurysm. Diagnosis is typically based on a physical exam and is supported by medical imaging such as a CT scan or MRI scan. A CT scan can rule out bleeding, but may not necessarily rule out ischemia, which early on typically does not show up on a CT scan. Other tests such as an electrocardiogram (ECG) and blood tests are done to determine risk factors and rule out other possible causes. Low blood sugar may cause similar symptoms.

Prevention includes decreasing risk factors, surgery to open up the arteries to the brain in those with problematic carotid narrowing, and warfarin in people with atrial fibrillation. Aspirin or statins may be recommended by physicians for prevention. A stroke or TIA often requires emergency care. An ischemic stroke, if detected within three to four and half hours, may be treatable with a medication that can break down the clot. Some hemorrhagic strokes benefit from surgery. Treatment to attempt recovery of lost function is called stroke rehabilitation, and ideally takes place in a stroke unit; however, these are not available in much of the world.

### Attribute Information
1) gender: "Male", "Female" or "Other"
2) age: age of the patient
3) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
4) heart disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease 
5) ever-married: "No" or "Yes"
6) worktype: "children", "Govtjov", "Neverworked", "Private" or "Self-employed" 
7) Residencetype: "Rural" or "Urban"
8) avgglucoselevel: average glucose level in blood
9) bmi: body mass index
10) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
11) stroke: 1 if the patient had a stroke or 0 if not

#### [Real data sources](https://www.kaggle.com/datasets/jillanisofttech/brain-stroke-dataset?sort=most-comments)

Modules

In [1]:
import pandas as pd
import sqlite3 as sq

Creating Database

In [3]:
# Creating Database
conn = sq.connect('brain_stroke.db')

# Create a table called brainStroke
query = """
CREATE TABLE brainStroke(
    gender VARCHAR(10),
    age DECIMAL(4,1),
    hypertension SMALLINT,
    heart_disease SMALLINT,
    ever_married VARCHAR(5),
    work_type VARCHAR(20),
    residence_type VARCHAR(10),
    avg_glucose_level DECIMAL(5,2),
    bmi DECIMAL(4,1),
    smoking_status VARCHAR(15),
    stroke SMALLINT
)
"""
with conn:
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS brainStroke")
    cur.execute(query)

In [5]:
# check our sqlite master for confirmation
query = "SELECT name FROM sqlite_master WHERE type='table'"
cur.execute(query)
cur.fetchall()

[('brainStroke',)]

Loading CSV File

In [6]:
brain_df = pd.read_csv('data/brain_stroke.csv')
brain_df.head()

In [8]:
csv_tuple = tuple(brain_df.itertuples(index=False, name=None))
csv_tuple[0]

('Male',
 67.0,
 0,
 1,
 'Yes',
 'Private',
 'Urban',
 228.69,
 36.6,
 'formerly smoked',
 1)

Insert values into database table

In [9]:
insert_query = "INSERT INTO brainStroke VALUES(?,?,?,?,?,?,?,?,?,?,?)"
cur.executemany(insert_query, csv_tuple)

In [10]:
query = "SELECT * FROM brainStroke LIMIT 5"
# cur.execute(query)
# cur.fetchall()

In [11]:
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


### Questions
1. What's the distribution of gender in the dataset?
2. What's the distribution of stroke in the dataset?
3. What's the distribution of brain stroke by gender?
4. For women, how many percent has stroke?
5. For men, how many percent has no stroke?
6. Details of the oldest people with stroke
7. How many people are married and has stroke?
8. How many people are not married and has stroke?
9. What's the minimum, average and maximum age in the data? 

1. What's the distribution of gender in the dataset?

In [12]:
query = """
SELECT    gender, COUNT(*) count
FROM      brainStroke
GROUP BY  gender
"""
pd.read_sql_query(query, conn)

,gender,count
0,Female,2907
1,Male,2074


2. What's the distribution of stroke in the dataset?

In [13]:
query = """
SELECT    stroke, COUNT(*) count
FROM      brainStroke
GROUP BY  stroke
"""
pd.read_sql_query(query, conn)

,stroke,count
0,0,4733
1,1,248


3. What's the distribution of brain stroke by gender.

In [14]:
query = """
SELECT    gender, stroke, COUNT(*) count
FROM      brainStroke
GROUP BY  gender, stroke
"""
pd.read_sql_query(query, conn)

,gender,stroke,count
0,Female,0,2767
1,Female,1,140
2,Male,0,1966
3,Male,1,108


4. For women, how many percent has stroke?

In [15]:
query = """
SELECT      gender, stroke, CAST(COUNT(*) AS FLOAT)/total.count * 100 AS Percent
FROM        brainStroke, (SELECT COUNT(*) count FROM brainStroke WHERE gender = 'Female') total
WHERE       gender = 'Female'
GROUP BY    stroke

UNION ALL

SELECT      gender, stroke, CAST(COUNT(*) AS FLOAT)/total.count * 100 AS Percent
FROM        brainStroke, (SELECT COUNT(*) count FROM brainStroke WHERE gender = 'Male') total
WHERE       gender = 'Male'
GROUP BY    stroke
"""
pd.read_sql_query(query, conn)

,gender,stroke,Percent
0,Female,0,95.184039
1,Female,1,4.815961
2,Male,0,94.792671
3,Male,1,5.207329


6. Details of the oldest people with stroke

In [16]:
query = """
WITH older_people AS (
    SELECT  *
    FROM    brainStroke
    WHERE   age > 70
)

SELECT  gender, work_type, COUNT(*) count 
FROM    older_people
GROUP BY    gender, work_type
"""
pd.read_sql_query(query, conn)

,gender,work_type,count
0,Female,Govt_job,52
1,Female,Private,193
2,Female,Self-employed,175
3,Male,Govt_job,31
4,Male,Private,143
5,Male,Self-employed,113


7. How many people are married and has stroke?

In [17]:
query = """
SELECT  COUNT(*) Married_and_has_stroke
FROM    brainStroke
WHERE   ever_married = 'Yes' AND stroke = 1
"""
pd.read_sql_query(query, conn)

,Married_and_has_stroke
0,219


8. How many people are not married and has stroke?

In [18]:
query = """
SELECT  COUNT(*) Not_Married_and_has_stroke
FROM    brainStroke
WHERE   ever_married = 'No' AND stroke = 1
"""
pd.read_sql_query(query, conn)

,Not_Married_and_has_stroke
0,29


9. What's the minimum, average and maximum age in the data? 

In [19]:
query = """
SELECT MAX(age) max_age, MIN(age) min_age, AVG(age) avg_age
  FROM brainStroke
"""
pd.read_sql_query(query, conn)

,max_age,min_age,avg_age
0,82,0.08,43.419859


In [20]:
query = """
SELECT *
  FROM brainStroke
 WHERE age < 10
 LIMIT 10
"""
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,3.00,0,0,No,children,Rural,95.12,18.0,Unknown,0
1,Female,8.00,0,0,No,Private,Urban,110.89,17.6,Unknown,0
2,Female,3.00,0,0,No,children,Urban,73.74,16.0,Unknown,0
3,Male,4.00,0,0,No,children,Rural,79.17,20.0,Unknown,0
4,Male,8.00,0,0,No,children,Rural,78.05,25.7,Unknown,0
5,Female,0.64,0,0,No,children,Urban,83.82,24.9,Unknown,0
6,Female,4.00,0,0,No,children,Rural,93.25,16.6,Unknown,0
7,Female,0.88,0,0,No,children,Rural,88.11,15.5,Unknown,0
8,Male,4.00,0,0,No,children,Rural,106.22,16.7,Unknown,0
9,Male,5.00,0,0,No,children,Rural,85.84,16.4,Unknown,0


In [24]:
query = """
SELECT  *
FROM    brainStroke
WHERE   work_type = 'children' AND (hypertension = 1 OR stroke =1)
"""
pd.read_sql_query(query, conn)

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Female,14.00,0,0,No,children,Rural,57.93,30.9,Unknown,1
1,Female,1.32,0,0,No,children,Urban,70.37,18.7,Unknown,1


### Summary
1. There are more Female than Men, 2,907 and 2,074 respectively
2. 4,733 persons have no Stroke while the remaining 248 have Stroke
4. 5% of Female has Stroke
5. There are more elderly people in the private sector than other work type
6. 219 Married people has Stroke
7. Two children has Stroke

Task

1. What's the distibution of hypertension in the dataset?
2. How many male, and female has hypertension?
3. For each smoking status show how many people have hypertension and how many do not?
4. How many male, and female has both hypertension and heart disease?
5. Which residence has the lowest number of people with no heart disease?
6. How many percent of people with private work type has no hypertension?